In [1]:
!pip install pandas numpy
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np

!pip install sklearn
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
train = pd.read_csv("../../data/Train.csv", sep=",")
metadata = pd.read_csv("../../data/metadata.csv", sep=",")
test = pd.read_csv("../../data/Test.csv", sep=",")

/home/nikhil/.local/share/virtualenvs/zindi_paygo-MEN1gcQP/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
metadata[metadata["ID"].isin(list(train["ID"].values))]

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_K00S4N4,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52
6,ID_6YU5YRB,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,56.0,Mount Kenya Region,Meru,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 15:15:16,2016-12-11 11:48:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37338,ID_OKJ96OW,2020-02-20 18:26:52,2400,NaN,0.0,FINANCED,DAILY,50,3,Female,38.0,Western,Bungoma,Farmer,d_light,240,14400.0,2020-10-17 18:26:52,2020-02-20 18:26:58,2020-12-19 07:16:44
37339,ID_5J9OMUP,2020-02-20 18:30:59,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,23.0,Nyanza,Siaya,Labourer,d_light,240,14400.0,2020-10-17 18:30:59,2020-02-20 18:31:06,2021-05-18 16:07:49
37340,ID_J4I8Z2V,2020-02-20 18:42:26,3000,NaN,0.0,FINANCED,DAILY,40,3,Male,31.0,Coast Region,Kwale,Teacher,d_light,340,16600.0,2021-01-25 18:42:26,2020-02-20 18:42:33,2021-04-21 15:59:37
37341,ID_GHHAQ9D,2020-02-20 18:46:30,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,27.0,NaN,UNKNOWN,Labourer,d_light,240,14400.0,2020-10-17 18:46:30,2020-02-20 18:46:37,2021-04-24 19:06:01


In [5]:
train.columns

Index(['ID', 'TransactionDates', 'PaymentsHistory', 'm1', 'm2', 'm3', 'm4',
       'm5', 'm6'],
      dtype='object')

In [6]:
train['TransactionDates'].values

array(["['04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018', '10-2018', '11-2018', '12-2018', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020']",
       "['04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '11-2018', '12-2018', '01-2019', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020']",
       "['02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020']",
       ...,
       "['01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020']",
       "['02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018']",
       "[

In [7]:
print(train["ID"].nunique())         # Each contract is unique
print(metadata["ID"].nunique())      # Each contract is unique

28007
37343


In [8]:
metadata.shape, train.shape

((37343, 20), (28007, 9))

In [9]:
train_merged = metadata.merge(train[['ID', 'TransactionDates', 'PaymentsHistory',
                      'm1', 'm2', 'm3', 'm4', 'm5', 'm6']], how="right", on="ID")

In [10]:
train_merged.shape

(28007, 28)

In [11]:
train_merged[train_merged["ID"] == "ID_K00S4N4"]["TransactionDates"].values

array(["['12-2015', '01-2016', '02-2016', '03-2016', '04-2016']"],
      dtype=object)

In [12]:
train_merged[train_merged["ID"] == "ID_K00S4N4"]["PaymentsHistory"].values

array(['[3050.0, 1050.0, 910.0, 1050.0, 1050.0]'], dtype=object)

In [13]:
train_merged[train_merged["ID"] == "ID_K00S4N4"][['m1', 'm2', 'm3', 'm4', 'm5', 'm6']].values

array([[1225., 1050., 1190.,  525., 1750., 1890.]])

In [14]:
train_merged.iloc[:, :-8]

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_MR53LEX,2018-04-23 12:28:05,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,NaN,Coast Region,Kwale,Teacher,d_light,556,33080.0,2019-10-31 12:28:05,2018-04-23 12:28:05,2021-05-26 17:14:45
1,ID_3D7NQUH,2018-04-17 10:27:35,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,26.0,South Rift,Narok,Farmer,d_light,556,33080.0,2019-10-25 10:27:35,2018-04-17 10:27:35,2021-05-26 16:54:53
2,ID_0IWQNPI,2020-02-18 14:23:44,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,21.0,Mount Kenya Region,Embu,Business,d_light,240,14400.0,2020-10-15 14:23:44,2020-02-18 14:23:48,2021-01-25 18:46:20
3,ID_IY8SYB9,2017-09-14 11:07:40,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,26.0,Mount Kenya Region,Meru,Farmer,d_light,364,16560.0,2018-09-13 11:07:40,2017-09-14 11:07:32,2018-12-20 16:50:39
4,ID_9XHL7VZ,2017-09-06 06:50:20,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,27.0,North Rift,West Pokot,Farmer,d_light,364,16560.0,2018-09-05 06:50:20,2017-09-06 06:50:16,2021-05-12 14:50:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28002,ID_CDZ24L9,2018-03-24 09:52:09,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,NaN,Coast Region,Kilifi,Labourer,d_light,556,33080.0,2019-10-01 09:52:09,2018-03-24 09:52:13,2019-10-11 12:37:41
28003,ID_0XINELS,2020-02-17 13:34:57,2400,NaN,0.0,FINANCED,DAILY,50,3,Female,NaN,Nairobi Region,Nairobi City,Business,d_light,240,14400.0,2020-10-14 13:34:57,2020-02-17 13:35:01,2021-03-13 14:23:22
28004,ID_PAU9JJU,2020-01-15 11:12:06,2000,NaN,0.0,FINANCED,DAILY,40,3,Male,29.0,South Rift,Kericho,Business,d_light,270,12800.0,2020-10-11 11:12:06,2020-01-15 11:12:11,2021-03-28 16:01:16
28005,ID_K866QHS,2018-02-25 16:51:21,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,57.0,Mount Kenya Region,Meru,Business,d_light,364,16560.0,2019-02-24 16:51:21,2018-02-25 16:53:11,2019-03-20 16:38:09


In [15]:
train_merged["SupplierName"].unique()

array(['d_light'], dtype=object)

In [16]:
train_merged["ID"].nunique()

28007

In [17]:
train_merged[train_merged.select_dtypes(exclude=['number']).columns]

,ID,RegistrationDate,UpsellDate,PaymentMethod,rateTypeEntity,MainApplicantGender,Region,Town,Occupation,SupplierName,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory
0,ID_MR53LEX,2018-04-23 12:28:05,NaN,FINANCED,DAILY,Male,Coast Region,Kwale,Teacher,d_light,2019-10-31 12:28:05,2018-04-23 12:28:05,2021-05-26 17:14:45,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0, 85.0..."
1,ID_3D7NQUH,2018-04-17 10:27:35,NaN,FINANCED,DAILY,Male,South Rift,Narok,Farmer,d_light,2019-10-25 10:27:35,2018-04-17 10:27:35,2021-05-26 16:54:53,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[2940.0, 970.0, 380.0, 880.0, 385.0, 440.0, 11..."
2,ID_0IWQNPI,2020-02-18 14:23:44,NaN,FINANCED,DAILY,Male,Mount Kenya Region,Embu,Business,d_light,2020-10-15 14:23:44,2020-02-18 14:23:48,2021-01-25 18:46:20,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2850.0, 1500.0, 1350.0, 610.0, 200.0, 250.0]"
3,ID_IY8SYB9,2017-09-14 11:07:40,NaN,FINANCED,DAILY,Female,Mount Kenya Region,Meru,Farmer,d_light,2018-09-13 11:07:40,2017-09-14 11:07:32,2018-12-20 16:50:39,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 780.0,..."
4,ID_9XHL7VZ,2017-09-06 06:50:20,NaN,FINANCED,DAILY,Male,North Rift,West Pokot,Farmer,d_light,2018-09-05 06:50:20,2017-09-06 06:50:16,2021-05-12 14:50:01,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2640.0, 910.0, 480.0, 280.0, 200.0, 180.0, 33..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28002,ID_CDZ24L9,2018-03-24 09:52:09,NaN,FINANCED,DAILY,Male,Coast Region,Kilifi,Labourer,d_light,2019-10-01 09:52:09,2018-03-24 09:52:13,2019-10-11 12:37:41,"['03-2018', '04-2018', '05-2018', '06-2018', '...","[5235.0, 1270.0, 940.0, 1370.0, 2020.0, 1750.0..."
28003,ID_0XINELS,2020-02-17 13:34:57,NaN,FINANCED,DAILY,Female,Nairobi Region,Nairobi City,Business,d_light,2020-10-14 13:34:57,2020-02-17 13:35:01,2021-03-13 14:23:22,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2980.0, 1000.0, 450.0, 1050.0, 850.0, 1250.0,..."
28004,ID_PAU9JJU,2020-01-15 11:12:06,NaN,FINANCED,DAILY,Male,South Rift,Kericho,Business,d_light,2020-10-11 11:12:06,2020-01-15 11:12:11,2021-03-28 16:01:16,"['01-2020', '02-2020', '03-2020', '04-2020', '...","[2580.0, 1020.0, 540.0, 360.0, 200.0, 660.0, 1..."
28005,ID_K866QHS,2018-02-25 16:51:21,NaN,FINANCED,DAILY,Female,Mount Kenya Region,Meru,Business,d_light,2019-02-24 16:51:21,2018-02-25 16:53:11,2019-03-20 16:38:09,"['02-2018', '03-2018', '04-2018', '05-2018', '...","[2000.0, 1160.0, 1240.0, 1120.0, 1000.0, 1200...."


In [18]:
train_merged.head()

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,...,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_MR53LEX,2018-04-23 12:28:05,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,...,2018-04-23 12:28:05,2021-05-26 17:14:45,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0, 85.0...",880.0,930.0,495.0,715.0,220.0,385.0
1,ID_3D7NQUH,2018-04-17 10:27:35,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,...,2018-04-17 10:27:35,2021-05-26 16:54:53,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[2940.0, 970.0, 380.0, 880.0, 385.0, 440.0, 11...",660.0,935.0,935.0,825.0,770.0,935.0
2,ID_0IWQNPI,2020-02-18 14:23:44,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,...,2020-02-18 14:23:48,2021-01-25 18:46:20,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2850.0, 1500.0, 1350.0, 610.0, 200.0, 250.0]",700.0,1350.0,1550.0,1400.0,1450.0,1200.0
3,ID_IY8SYB9,2017-09-14 11:07:40,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,...,2017-09-14 11:07:32,2018-12-20 16:50:39,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 780.0,...",580.0,480.0,800.0,1260.0,1650.0,530.0
4,ID_9XHL7VZ,2017-09-06 06:50:20,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,...,2017-09-06 06:50:16,2021-05-12 14:50:01,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2640.0, 910.0, 480.0, 280.0, 200.0, 180.0, 33...",40.0,440.0,460.0,360.0,80.0,330.0


In [19]:
from utils import split, length_calc, sum_calc, mean_calc, median_calc, max_calc, min_calc, std_dev_calc
def feature_construction(df):
    try:
        df["SplitPaymentsHistory"] = df.apply(lambda row: split(row["PaymentsHistory"]), axis=1)
        df["SplitTransactionDates"] = df.apply(lambda row: split(row["TransactionDates"]), axis=1)
        df["nb_payments"] = df.apply(lambda row: length_calc(row["SplitPaymentsHistory"]), axis=1)
        df["amount_paid"] = df.apply(lambda row: sum_calc(row["SplitPaymentsHistory"]), axis=1)
        df["percent_amt_paid"] = df["amount_paid"] / train_merged["TotalContractValue"]
        df["mean_amt_paid"] = df.apply(lambda row: mean_calc(row["SplitPaymentsHistory"]), axis=1)
        df["median_amt_paid"] = df.apply(lambda row: median_calc(row["SplitPaymentsHistory"]), axis=1)
        df["max_amt_paid"] = df.apply(lambda row: max_calc(row["SplitPaymentsHistory"]), axis=1)
        df["min_amt_paid"] = df.apply(lambda row: min_calc(row["SplitPaymentsHistory"]), axis=1)
        df["stddev_amt_paid"] = df.apply(lambda row: std_dev_calc(row["SplitPaymentsHistory"]), axis=1)
    except KeyError as e:
        raise Exception("Column Missing")
        
    return df

train_merged = feature_construction(train_merged)

In [20]:
train_merged.head()

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,...,SplitPaymentsHistory,SplitTransactionDates,nb_payments,amount_paid,percent_amt_paid,mean_amt_paid,median_amt_paid,max_amt_paid,min_amt_paid,stddev_amt_paid
0,ID_MR53LEX,2018-04-23 12:28:05,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,...,"[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0,...","['04-2018', '05-2018', '06-2018', '07-2018'...",31,16035.0,0.484734,517.26,350.0,3600.0,55.0,652.49
1,ID_3D7NQUH,2018-04-17 10:27:35,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,...,"[2940.0, 970.0, 380.0, 880.0, 385.0, 440....","['04-2018', '05-2018', '06-2018', '07-2018'...",30,22136.0,0.669166,737.87,655.0,2940.0,380.0,452.03
2,ID_0IWQNPI,2020-02-18 14:23:44,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,...,"[2850.0, 1500.0, 1350.0, 610.0, 200.0, 25...","['02-2020', '03-2020', '04-2020', '05-2020'...",6,6760.0,0.469444,1126.67,980.0,2850.0,200.0,1005.32
3,ID_IY8SYB9,2017-09-14 11:07:40,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,...,"[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 7...","['09-2017', '10-2017', '11-2017', '12-2017'...",10,11260.0,0.679952,1126.00,1140.0,2200.0,380.0,511.30
4,ID_9XHL7VZ,2017-09-06 06:50:20,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,...,"[2640.0, 910.0, 480.0, 280.0, 200.0, 180....","['09-2017', '10-2017', '11-2017', '12-2017'...",22,7779.0,0.469746,353.59,190.0,2640.0,40.0,559.24


In [21]:
train_merged.columns

Index(['ID', 'RegistrationDate', 'Deposit', 'UpsellDate', 'AccessoryRate',
       'PaymentMethod', 'rateTypeEntity', 'RatePerUnit', 'DaysOnDeposit',
       'MainApplicantGender', 'Age', 'Region', 'Town', 'Occupation',
       'SupplierName', 'Term', 'TotalContractValue', 'ExpectedTermDate',
       'FirstPaymentDate', 'LastPaymentDate', 'TransactionDates',
       'PaymentsHistory', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6',
       'SplitPaymentsHistory', 'SplitTransactionDates', 'nb_payments',
       'amount_paid', 'percent_amt_paid', 'mean_amt_paid', 'median_amt_paid',
       'max_amt_paid', 'min_amt_paid', 'stddev_amt_paid'],
      dtype='object')

### We can ignore UpSell since it represents only 2.7% of the rows

In [22]:
train_merged.UpsellDate.count() / train_merged.shape[0] * 100

2.6493376655836043

### Drop irrelevant columns

In [23]:
def drop_cols(df):
    df.drop(
        [
            "ID",
            "UpsellDate",
            "PaymentMethod",
            "TransactionDates",
            "PaymentsHistory",
            "SupplierName",
            "Town"
        ],
        inplace=True,
        axis=1
    )
drop_cols(train_merged)

In [24]:
train_merged.head()

,RegistrationDate,Deposit,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Occupation,...,SplitPaymentsHistory,SplitTransactionDates,nb_payments,amount_paid,percent_amt_paid,mean_amt_paid,median_amt_paid,max_amt_paid,min_amt_paid,stddev_amt_paid
0,2018-04-23 12:28:05,2500,0.0,DAILY,55,3,Male,NaN,Coast Region,Teacher,...,"[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0,...","['04-2018', '05-2018', '06-2018', '07-2018'...",31,16035.0,0.484734,517.26,350.0,3600.0,55.0,652.49
1,2018-04-17 10:27:35,2500,0.0,DAILY,55,3,Male,26.0,South Rift,Farmer,...,"[2940.0, 970.0, 380.0, 880.0, 385.0, 440....","['04-2018', '05-2018', '06-2018', '07-2018'...",30,22136.0,0.669166,737.87,655.0,2940.0,380.0,452.03
2,2020-02-18 14:23:44,2400,0.0,DAILY,50,3,Male,21.0,Mount Kenya Region,Business,...,"[2850.0, 1500.0, 1350.0, 610.0, 200.0, 25...","['02-2020', '03-2020', '04-2020', '05-2020'...",6,6760.0,0.469444,1126.67,980.0,2850.0,200.0,1005.32
3,2017-09-14 11:07:40,2000,0.0,DAILY,40,7,Female,26.0,Mount Kenya Region,Farmer,...,"[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 7...","['09-2017', '10-2017', '11-2017', '12-2017'...",10,11260.0,0.679952,1126.00,1140.0,2200.0,380.0,511.30
4,2017-09-06 06:50:20,2000,0.0,DAILY,40,7,Male,27.0,North Rift,Farmer,...,"[2640.0, 910.0, 480.0, 280.0, 200.0, 180....","['09-2017', '10-2017', '11-2017', '12-2017'...",22,7779.0,0.469746,353.59,190.0,2640.0,40.0,559.24


### Feature Encoding

In [25]:
# OHE

def encode_and_bind(df, feature_name):
    dummies = pd.get_dummies(df[[feature_name]])
    res = pd.concat([df, dummies], axis=1)
    res = res.drop([feature_name], axis=1)
    return(res)

for feature in ["MainApplicantGender", "Region", "Occupation", "rateTypeEntity"]:
    train_merged = encode_and_bind(train_merged, feature)

In [26]:
train_merged.head()

,RegistrationDate,Deposit,AccessoryRate,RatePerUnit,DaysOnDeposit,Age,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,...,Occupation_Business,Occupation_Driver/Motorbike Rider,Occupation_Farmer,Occupation_Government Employee,Occupation_Labourer,Occupation_Other,Occupation_Teacher,rateTypeEntity_DAILY,rateTypeEntity_MONTHLY,rateTypeEntity_WEEKLY
0,2018-04-23 12:28:05,2500,0.0,55,3,NaN,556,33080.0,2019-10-31 12:28:05,2018-04-23 12:28:05,...,0,0,0,0,0,0,1,1,0,0
1,2018-04-17 10:27:35,2500,0.0,55,3,26.0,556,33080.0,2019-10-25 10:27:35,2018-04-17 10:27:35,...,0,0,1,0,0,0,0,1,0,0
2,2020-02-18 14:23:44,2400,0.0,50,3,21.0,240,14400.0,2020-10-15 14:23:44,2020-02-18 14:23:48,...,1,0,0,0,0,0,0,1,0,0
3,2017-09-14 11:07:40,2000,0.0,40,7,26.0,364,16560.0,2018-09-13 11:07:40,2017-09-14 11:07:32,...,0,0,1,0,0,0,0,1,0,0
4,2017-09-06 06:50:20,2000,0.0,40,7,27.0,364,16560.0,2018-09-05 06:50:20,2017-09-06 06:50:16,...,0,0,1,0,0,0,0,1,0,0


In [27]:
train_merged.drop(["RegistrationDate", "ExpectedTermDate", "FirstPaymentDate", "LastPaymentDate", "SplitPaymentsHistory", "SplitTransactionDates", "m2", "m3", "m4", "m5", "m6"], inplace=True, axis=1)


### Data Leakage here. Will be handled later.

In [28]:
train_merged.Age = train_merged.Age.fillna(train_merged.Age.mean())

In [29]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

np.random.seed(0)
def predict_model(train_array, target):
    model = xgb.XGBRegressor(use_label_encoder=False,
                             verbosity=0,
                             objective="reg:squarederror",
                             reg_lambda=0,
                             reg_alpha=50)
    for i in range(1, 11):
        X_train, X_test, y_train, y_test = train_test_split(train_array, target, test_size=0.20)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        feature_importance = model.feature_importances_
        mape = mean_absolute_percentage_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = pow(mse, 0.5)

    return y_pred, mape, mse, feature_importance, rmse

predict_model(train_merged.drop(["m1"], axis=1), train_merged["m1"])

(array([ 415.52536,  817.64044,  454.33908, ..., 1139.8318 ,  797.1174 ,
         606.1118 ], dtype=float32),
 1.3810772757316307,
 214496.1800167461,
 array([0.00477186, 0.24319133, 0.00956253, 0.02578597, 0.02451435,
        0.02955278, 0.0465116 , 0.01390662, 0.03766868, 0.03131246,
        0.20227723, 0.06542175, 0.00839889, 0.01759385, 0.01235598,
        0.00641496, 0.        , 0.01137408, 0.01511769, 0.01284616,
        0.00541333, 0.01150084, 0.0855708 , 0.00899474, 0.00689266,
        0.00840084, 0.00800553, 0.01273176, 0.00798615, 0.01231058,
        0.01361401, 0.        , 0.        , 0.        ], dtype=float32),
 463.1373230660061)